I think with the time frame my best approach is to use linear optimisation. It won't provide the highest output, but good enough for the sake of this exercise. The plan would be this: 

1) Change OfferAmt and optimise for the highest output of Spend
2) Use Redemption Probability and median SpendPerCoupon£ to get the final spend values. 

REMEMBER: We can only offer an amout they have already been offered. The SUM of Coupons Redeemed cannot be more than £30,000

__Dataset__

Data needs to be filtered to include the target date week.

- Gradient: Measure sensitiity to price increase of coupon.
- Predicted_probaility: Probability they will take the coupon anount
- Max: Max Coupon they have ever redeemed
- Min: Min Coupon they have ever redeemed
- SpendPerOfferAmtRedeemed: The median Spend per coupon recieved


In [227]:
import pandas as pd
from datetime import datetime, timedelta

In [228]:
df_training = pd.read_csv('../data/HistoricalTrainingData.csv')

# Filter customers who visited the store in the last 90 days from 2019-07-01
cutoff_date = pd.to_datetime('2019-07-01') - pd.Timedelta(days=90)
df_recent_visitors = df_training[df_training['TripDate'] >= cutoff_date.strftime('%Y-%m-%d')]

# Extract unique PatronIDs of recent visitors
recent_visitor_ids = df_recent_visitors['PatronID'].unique()

# Load customer prediction metrics and sensitivity metrics
df_customer_prediction_metrics = pd.read_csv('./customer_prediction_metrics_nn.csv')
df_customer_sensitivity_metrics = pd.read_csv('./customer_sensitivity_metrics.csv')

df_customer_prediction_metrics['UseStartDate'] = pd.to_datetime(df_customer_prediction_metrics['UseStartDate'])
df_customer_prediction_metrics = df_customer_prediction_metrics[df_customer_prediction_metrics["UseStartDate"] == pd.to_datetime("2019-07-01")]

# Merge the two dataframes on PatronID
df_merged_metrics = pd.merge(df_customer_prediction_metrics, df_customer_sensitivity_metrics, on='PatronID', how='inner')

# Filter by the list of recent visitor PatronIDs
data = df_merged_metrics[df_merged_metrics['PatronID'].isin(recent_visitor_ids)]

In [229]:
# Calculate the ratio of Spend to OfferAmtRedeemed for transactions where OfferAmtRedeemed > 0
df_training['SpendPerOfferAmt'] = df_training.loc[df_training['OfferAmt'] > 0, 'Spend'] / df_training.loc[df_training['OfferAmt'] > 0, 'OfferAmt']


# Calculate total visits by customer in the training data
total_visits_by_customer = df_training.groupby('PatronID').size().reset_index(name='TotalVisits')

# Calculate the median SpendPerOfferAmt for each customer
median_spend_per_offer_amt_per_customer = df_training.groupby('PatronID')['SpendPerOfferAmt'].median().reset_index()
median_spend_per_offer_amt_per_customer.rename(columns={'SpendPerOfferAmt': 'MedianSpendPerOfferAmt'}, inplace=True)

# Calculate the maximum spend per individual in the training data
max_spend_per_individual = df_training.groupby('PatronID')['Spend'].max().reset_index()
max_spend_per_individual.rename(columns={'Spend': 'MaxSpendInTraining'}, inplace=True)

# Calculate the median of the Spend to OfferAmtRedeemed ratio for each customer
median_spend_per_offer_redeemed_per_customer = df_training.groupby('PatronID')['Spend'].median().reset_index()
median_spend_per_offer_redeemed_per_customer.rename(columns={'Spend': 'MedianSpendInTraining'}, inplace=True)

# Calculate the median of the Spend to OfferAmtRedeemed ratio for each customer
min_spend_per_offer_redeemed_per_customer = df_training.groupby('PatronID')['Spend'].min().reset_index()
min_spend_per_offer_redeemed_per_customer.rename(columns={'Spend': 'MinSpendInTraining'}, inplace=True)



# Calculate the maximum and minimum OfferedAmt for each customer
offer_ranges = df_training.groupby('PatronID')['OfferAmt'].agg(['max', 'min']).reset_index()
offer_ranges.rename(columns={'max': 'MaxOfferAmt', 'min': 'MinOfferAmt'}, inplace=True)

# Proceed with other merges as before
optimisation_updated = data.merge(offer_ranges, on='PatronID', how='left')\
                            .merge(total_visits_by_customer, on='PatronID', how='left')\
                            .merge(max_spend_per_individual, on='PatronID', how='left')\
                            .merge(median_spend_per_offer_redeemed_per_customer, on='PatronID', how='left')\
                            .merge(min_spend_per_offer_redeemed_per_customer, on='PatronID', how='left')

# Merge the median spend per offer amount redeemed with the optimization dataset
optimised_data = optimisation_updated.merge(median_spend_per_offer_amt_per_customer, on='PatronID', how='left')

# Linear Optimisation

In [230]:
from scipy.optimize import linprog
import numpy as np

In [231]:
optimised_data['Spend'].fillna(0, inplace=True)
optimised_data['OfferAmt'].fillna(0, inplace=True)
optimised_data['OfferAmtRedeemed'].fillna(0, inplace=True)

In [232]:
# Define a probability threshold
PROBABILITY_THRESHOLD_LOW = 0.2 # min-median spend in training
PROBABILITY_THRESHOLD_MED = 0.5 # The median spend in training
PROBABILITY_THRESHOLD_HIGH = 0.6 # The median spend in training
PROBABILITY_THRESHOLD_VHIGH = 0.7 # Max spend in training
[InternetShortcut]
URL=https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

# Preprocessing: Replace inf, NaN, or None values with the median for each column
for column in optimised_data.columns:
    if optimised_data[column].dtype in ['float64', 'int64']:
        median_value =optimised_data[column].replace([np.inf, -np.inf], np.nan).median()
        optimised_data[column].replace([np.inf, -np.inf], np.nan, inplace=True)
        optimised_data[column].fillna(median_value, inplace=True)

# Calculate maximum offer amount based on max historical spend
optimised_data['MaxOfferBasedOnMaxSpend'] = optimised_data['MaxSpendInTraining'] / optimised_data['MedianSpendPerOfferAmt']

# Adjust the bounds for the optimization problem
bounds = []
for index, row in optimised_data.iterrows():
    # Set initial lower and upper bounds from MinOfferAmt and MaxOfferAmt
    lower_bound, upper_bound = row['MinOfferAmt'], row['MaxOfferAmt']
    
    # Adjust the upper bound based on probability thresholds
    if row['Predicted_Probability'] >= PROBABILITY_THRESHOLD_VHIGH:
        upper_bound = min(upper_bound, row['MaxSpendInTraining'] / row['MedianSpendPerOfferAmt'])
    elif row['Predicted_Probability'] >= PROBABILITY_THRESHOLD_HIGH:
        # Set upper bound to not exceed a certain percentage of MaxSpendInTraining
        upper_bound = min(upper_bound, 0.75 * row['MaxSpendInTraining'] / row['MedianSpendPerOfferAmt'])
    elif row['Predicted_Probability'] >= PROBABILITY_THRESHOLD_MED:
        # Further constrain the upper bound for medium probabilities
        upper_bound = min(upper_bound, 0.5 * row['MaxSpendInTraining'] / row['MedianSpendPerOfferAmt'])
    elif row['Predicted_Probability'] >= PROBABILITY_THRESHOLD_LOW:
        # Only allow spending up to the median spend for the lowest tier
        upper_bound = min(upper_bound, row['MedianSpendInTraining'] / row['MedianSpendPerOfferAmt'])
    else:
        # No offer for probabilities below the lowest threshold
        lower_bound, upper_bound = 0, 0
    
    # Ensure lower bound does not exceed the new upper bound
    if lower_bound > upper_bound:
        adjusted_bound = (upper_bound, upper_bound)
    else:
        adjusted_bound = (lower_bound, upper_bound)
    
    bounds.append(adjusted_bound)
    
# Set up the optimization problem
c = -optimised_data['MedianSpendPerOfferAmt'].values  # Objective function coefficients (negative for maximization)
A = [[1] * len(data)]  # Constraint coefficients for the budget
b = [30000]  # Total budget constraint

# Solve the optimization problem
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

if res.success:
    # Insert the "OptimizedOfferAmount" into the DataFrame
    optimised_data['OptimizedOfferAmount'] = res.x
    
    # Calculate "PredictedSpend" based on the "OptimizedOfferAmount"
    optimised_data['PredictedSpend'] = optimised_data['OptimizedOfferAmount'] * optimised_data['MedianSpendPerOfferAmt']
    optimised_data['PredictedSpend'] = np.where(optimised_data['Predicted_Probability'] >= PROBABILITY_THRESHOLD,
                                      optimised_data['PredictedSpend'], 0)

    # Display the total expected spend and offer amount based on optimization
    total_expected_spend = -res.fun
    total_offer_amount = sum(res.x)
    
    print("Optimization successful!")
    print(f"Total Expected Spend: £{total_expected_spend:.2f}")
    print(f"Total Offer Amount: £{total_offer_amount:.2f}")
    print(f"ROI: {(total_expected_spend/total_offer_amount):.2f}")
else:
    print("Optimization failed:", res.message)


Optimization successful!
Total Expected Spend: £74494.66
Total Offer Amount: £30000.00
ROI: 2.48


In [233]:
print(f"Total Actual Spend: £{data['Spend'].sum():.2f}")
print(f"Total Actual Offered Amount: £{data['OfferAmt'].sum():.2f}")
print(f"ROI: {(data['Spend'].sum()/data['OfferAmt'].sum()):.2f}")

Total Actual Spend: £223698.54
Total Actual Offered Amount: £110907.00
ROI: 2.02


In [234]:
# result = optimised_data[['PatronID', 'OfferAmt', 'Spend', 'RedeemedYN', 'Predicted_Probability', 'Predicted_RedeemedYN', 'Gradient', 'MedianSpendPerOfferAmt', 'TotalVisits','OptimizedOfferAmount', 'MaxOfferAmt', 'MaxSpendInTraining', 'PredictedSpend']]
# result.to_csv("../result/nn_lr_coupon_strat.csv", index=False)

# Result

The output ROI looks good, I am happy with my output. I will use this model to finish the problem. However:

1) This model does not take into account the changing probability based on offers
2) This is a linear problem solve, therefore it can't take into account the changing redepmtion probability if we tried
3) This model is also too focused on the high earners. We need a more risk averse approach.